# 3D Rotation
3D Rotation can be represented in different ways as discussed in last week's lectures.
These include:
1.	Rotation matrices
2.  Three angle representation
3.	Two vector representation
4.	Rotation about an eigen vector
5.	Unit quaternion
6.	Matrix exponential

We explored the `rotation matrices` approach in last weeks lab exercise.

This lab is designed for you to implement rotation matrices in each of those different methods.

In [12]:
# import required libraries
import numpy as np
import matplotlib.pyplot as plt
# To render plots inline
%matplotlib inline

# For unit tests
from tester import LabTester
lab_tester = LabTester()

In [13]:
def rotation_three_angle(alpha:float, gamma:float, theta:float, p:np.ndarray) -> np.ndarray:
    """
    Input:
        alpha, gamma, theta - Euler angles of rotation about the axes (x, y, x)
        p                   - The vector to be transformed
    Output: resulting coordinates after appyling the rotation
    """
    # Your code here - Use Euler angles method => (x,y,x)
    alpha = np.radians(alpha)
    gamma = np.radians(gamma)
    theta = np.radians(theta)

    Rx1 = np.array([[1,0,0], [0, np.cos(alpha), -np.sin(alpha)], [0, np.sin(alpha), np.cos(alpha)]])
    Ry = np.array([[np.cos(gamma), 0, np.sin(gamma)], [0, 1, 0], [-np.sin(gamma), 0, np.cos(gamma)]])
    Rx2 = np.array([[1,0,0], [0, np.cos(theta), -np.sin(theta)], [0, np.sin(theta), np.cos(theta)]])    
    
    R = Rx1 @ Ry @ Rx2
    p = R @ p
    return p

In [14]:
# Run test cases
lab_tester.test_rotation_three_angle(rotation_three_angle)


=== Testing Three Angle Rotation Implementation ===
✅ No Rotation
✅ 45 Degree Rotation
✅ Full Rotation
✅ Negative 45 Degree Rotation


In [15]:
import numpy as np

def rotation_two_vector(a_v: np.ndarray, o_v: np.ndarray, p: np.ndarray) -> np.ndarray:
    """
    Rotates a point 'p' so that the 'a_v' (approach vector) aligns with 'o_v' (orientation vector)
    using the two-vector representation method.

    Parameters:
        a_v (np.ndarray): Approach vector (initial direction).
        o_v (np.ndarray): Orientation vector (desired direction).
        p (np.ndarray): The point to be transformed.

    Returns:
        np.ndarray: Transformed coordinates after applying the rotation.
    """
    # Normalize vectors
    a_v = a_v / np.linalg.norm(a_v)
    o_v = o_v / np.linalg.norm(o_v)

    angle = np.arccos(np.clip(np.dot(a_v, o_v), -1.0, 1.0))
    if angle == 0:
        return p
    if angle == np.pi:
        return -p

    # Compute the normal vector (perpendicular to both a_v and o_v)
    n = np.cross(o_v, a_v)
    n_norm = np.linalg.norm(n)

    n /= n_norm  # Normalize the normal vector

    o_v = np.cross(a_v, n)  # Update orientation vector
    o_v /= np.linalg.norm(o_v)  # Normalize the orientation vector



    R = np.array([n, o_v, a_v]).T  # Rotation matrix

    # Apply transformation
    rotated_p = R @ p

    return rotated_p


In [16]:
lab_tester.test_rotation_two_vector(rotation_two_vector)


=== Testing Two Vector Rotation Implementation ===
✅ X-Axis Rotation
✅ Y-Axis Rotation
✅ Z-Axis Rotation
✅ 45-degree Z Rotation


In [17]:
def rotation_eigen_vector(neta: np.ndarray, theta:float, p:np.ndarray) -> np.ndarray:
    """
    Input:
        neta    - Eigen vector with eigen value 1
        theta   - Angle of rotation about neta
        p       - The vector to be transformed
    Output: resulting coordinates after appyling the rotation
    """
    # Your code here - Use Rodirgues Formula
    theta = np.radians(theta)
    neta = neta / np.linalg.norm(neta)
    I = np.eye(3)
    neta_x = np.array([[0, -neta[2], neta[1]], [neta[2], 0, -neta[0]], [-neta[1], neta[0], 0]])
    R = I + np.sin(theta) * neta_x + (1 - np.cos(theta)) * np.dot(neta_x, neta_x)
    p = np.dot(R, p)
    return p

In [18]:
lab_tester.test_rotation_eigen_vector(rotation_eigen_vector)


=== Testing Eigen Vector Rotation Implementation ===
✅ No Rotation
✅ 180-degree Z Rotation
✅ 90-degree X Rotation
✅ 45-degree Y Rotation
✅ Negative 90-degree Z Rotation


In [19]:
def quaternion_multiply(q1, q2):
    """Multiplies two quaternions."""
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2
    return np.array([
        w1*w2 - x1*x2 - y1*y2 - z1*z2,
        w1*x2 + x1*w2 + y1*z2 - z1*y2,
        w1*y2 - x1*z2 + y1*w2 + z1*x2,
        w1*z2 + x1*y2 - y1*x2 + z1*w2
    ])


def rotation_unit_quaternion(q: np.ndarray, p: np.ndarray) -> np.ndarray:
    # Your code here - q: Unit quaternion vector [Real, i, j, k]
    # Your code here - p: The vector to be transformed
    q = q / np.linalg.norm(q)
    
    # Convert vector p to quaternion form with zero scalar part
    p_q = np.concatenate(([0], p))
    
    # Compute the conjugate of q
    q_conj = np.array([q[0], -q[1], -q[2], -q[3]])
    
    # Rotate p: p' = q * p_q * q_conj
    q_p = quaternion_multiply(q, p_q)
    q_p_q_conj = quaternion_multiply(q_p, q_conj)
    
    # Return the vector part of the resulting quaternion
    return q_p_q_conj[1:]

In [20]:
lab_tester.test_rotation_unit_quaternion(rotation_unit_quaternion)


=== Testing Unit Quaternion Rotation Implementation ===
✅ No Rotation
✅ 90-degree X Rotation
✅ 180-degree Y Rotation
✅ 45-degree Z Rotation
✅ Negative 90-degree X Rotation


In [21]:
def skew_symmetric(w_x: np.ndarray) -> np.ndarray:
    """Returns the skew-symmetric matrix of a 3D vector w_x."""
    return np.array([
        [0, -w_x[2], w_x[1]],
        [w_x[2], 0, -w_x[0]],
        [-w_x[1], w_x[0], 0]
    ])

def rotation_matrix_exp(w_x : np.ndarray, theta: float, p: np.ndarray) -> np.ndarray:
    # Your code here
    # w_x: The rotation axis
    # theta: The angle of rotation
    # p: The vector to be transformed
    # Convert angle to radians
    theta_rad = np.radians(theta)

    # Compute the skew-symmetric matrix of w_x
    W_x = skew_symmetric(w_x)
    
    # Calculate the matrix exponential using the Rodriguez formula
    R = np.eye(3) + np.sin(theta_rad) * W_x + (1 - np.cos(theta_rad)) * np.dot(W_x, W_x)
    
    # Apply the rotation to the point
    p_rotated = np.dot(R, p)
    
    return p_rotated

In [22]:
lab_tester.test_rotation_matrix_exp(rotation_matrix_exp)


=== Testing Matrix Exponential Rotation Implementation ===
✅ No Rotation
✅ 90-degree X-axis Rotation
✅ 90-degree Y-axis Rotation
✅ 90-degree Z-axis Rotation
✅ 180-degree X-axis Rotation
✅ 180-degree Y-axis Rotation
✅ 180-degree Z-axis Rotation
✅ 45-degree X-axis Rotation
